This notebook shows how to plot the psychometric curve of the data in a session from the IBL DataJoint database

Here we first import some modules from the ibl pipeline and some other modules

In [ ]:
# behavior data
from ibl_pipeline import behavior

# analyzed result of behavioral data
from ibl_pipeline.analyses import behavior as behavior_analyses

# meta information of the subject and session
from ibl_pipeline import subject, acquisition

# Function to perform the model fits of the psychometric function
from ibl_pipeline.utils import psychofit as psy

# some regular modules
import numpy as np
import datetime
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

The fit results of the psychometric curve are saved in the table `behavior_analyses.PsychResultsBlock`, we can browse entries in the table for a particular subject, the prob_left_block marks which prior of probability left have been used in the block

In [ ]:
q = behavior_analyses.PsychResultsBlock & (subject.Subject & 'subject_nickname="CSHL_015"')

Let's check the results of sessions collected after 2019-09-15, by:

In [ ]:
q & 'session_start_time > "2019-09-15"'

Let's plot the psychometric curve of the last session on 9/17, containing three blocks with `prob_left` 0.2, 0.5 and 0.8

In [ ]:
psych_results = q & {'session_start_time': datetime.datetime(2019, 9, 16, 13, 44, 46)}

Now let's fetch the fit parameters and behavioral statistics from the table as a list of dictionaries.

In [ ]:
dict_results = psych_results.fetch(
    'signed_contrasts', 'prob_choose_right', 'n_trials_stim', 'n_trials_stim_right',
    'threshold', 'bias', 'lapse_low', 'lapse_high', as_dict=True)

Next let's loop through the blocks and plot the psychometric curves:

In [ ]:
colors = [[1., 0., 0.], [0., 0., 0.], [0., 0., 1.]]
for result, color in zip(dict_results, colors):
    pars = [result['bias'], result['threshold'], result['lapse_low'], result['lapse_high']]
    contrasts = result['signed_contrasts'] * 100
    contrasts_fit = np.arange(-100, 100)
    prob_right_fit = psy.erf_psycho_2gammas(pars, contrasts_fit) *100
    sns.lineplot(contrasts_fit, prob_right_fit, color=color)
    sns.lineplot(x=contrasts, y=result['prob_choose_right']*100, err_style="bars", linewidth=0, linestyle='None', mew=0.5,
            marker='.', ci=68, color=color)

plt.gca().set_xlabel('Signed Contrast (%)')
plt.gca().set_ylabel('Rightward Choice (%)')